# Customer Purchase Behavior — EDA, Cleaning, and Feature Engineering

This notebook is designed to run **top-to-bottom** with minimal edits.

## What it does
1. Loads the dataset reliably (handles common CSV issues).
2. Validates structure (shape, columns, dtypes, basic stats).
3. EDA focused on **customer purchase prediction**:
   - Missing values (table + bar chart)
   - Data types / mixed types
   - Duplicates
   - Outliers **only for customer-relevant numeric columns**
4. Preprocessing:
   - Fixes missing tokens and trims strings
   - Parses date/time columns when applicable
   - Drops constant and ID-like columns
   - Flags and (optionally) drops likely **leakage/irrelevant** columns (e.g., company profit)
   - Imputes missing values (median/mode)
   - Caps outliers (IQR)
5. Feature engineering:
   - Datetime features
   - Frequency encodings for categoricals
   - Log transforms for skewed numeric columns
6. Saves:
   - `customers_cleaned.csv`
   - `customers_features.csv` (ML-ready, one-hot encoded)

> Tip for team discussion: This notebook also prints a **Decision Log** so you can explain what changed and why.


In [ ]:
# ============================================================
# 0) Imports & Settings
# ============================================================
import os
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 180)
plt.rcParams["figure.figsize"] = (10, 4)

# ---- Set your CSV path here ----
# (A) If running in this sandbox environment, the default below will work.
# (B) If running locally on Windows, set something like:
# DATA_PATH = r"C:\Users\lawre\Downloads\customers_data (1).csv"
DATA_PATH = r"/mnt/data/customers_data (1).csv"

OUT_DIR = r"./eda_outputs"
os.makedirs(OUT_DIR, exist_ok=True)

CLEAN_PATH    = os.path.join(OUT_DIR, "customers_cleaned.csv")
FEATURES_PATH = os.path.join(OUT_DIR, "customers_features.csv")
REPORT_PATH   = os.path.join(OUT_DIR, "eda_decision_log.txt")

# Optional: if you KNOW your target column name, set it.
# Example: TARGET_COL = "Purchased"
TARGET_COL = None


In [ ]:
# ============================================================
# 1) Robust CSV Loader (fixes most read errors)
# ============================================================
def robust_read_csv(path: str) -> pd.DataFrame:
    if not os.path.exists(path):
        raise FileNotFoundError(
            f"File not found: {path}\n"
            "Fix: update DATA_PATH to your real file location."
        )
    attempts = [
        {"encoding": "utf-8", "sep": ","},
        {"encoding": "utf-8-sig", "sep": ","},
        {"encoding": "cp1252", "sep": ","},
        {"encoding": "latin1", "sep": ","},
        {"encoding": "utf-8", "sep": ";"},
        {"encoding": "cp1252", "sep": ";"},
    ]
    last_err = None
    for opts in attempts:
        try:
            df = pd.read_csv(path, **opts)
            # If it became 1 column, likely wrong separator; keep trying
            if df.shape[1] == 1 and opts["sep"] == ",":
                continue
            print(f"✅ Loaded with encoding={opts['encoding']} sep='{opts['sep']}' | shape={df.shape}")
            return df
        except Exception as e:
            last_err = e
    raise RuntimeError(f"Could not read CSV. Last error: {last_err}")

df_raw = robust_read_csv(DATA_PATH)
df_raw.head(10)


In [ ]:
# ============================================================
# 2) Validate Structure
# ============================================================
print("Shape:", df_raw.shape)
print("\nColumns:")
print(list(df_raw.columns))

print("\nInfo:")
df_raw.info()

print("\nBasic numeric stats:")
display(df_raw.describe(include=[np.number]).T.head(20))

print("\nBasic categorical stats (top columns):")
cat_cols = df_raw.select_dtypes(include=["object"]).columns.tolist()
display(df_raw[cat_cols].describe().T.head(20) if len(cat_cols) else "No object columns")


In [ ]:
# ============================================================
# 3) EDA (Focused for Purchase Prediction)
# ============================================================

# 3.1 Missing values
missing = df_raw.isna().sum().sort_values(ascending=False)
missing_nonzero = missing[missing > 0]
display(missing_nonzero if len(missing_nonzero) else "✅ No missing values detected.")

# Bar plot of missing values
if len(missing_nonzero):
    plt.figure(figsize=(10, 4))
    missing_nonzero.head(30).plot(kind="bar")
    plt.title("Missing Values per Column (Top 30)")
    plt.ylabel("Count missing")
    plt.tight_layout()
    plt.show()

# 3.2 Duplicates
dup_count = df_raw.duplicated().sum()
print("Duplicate rows:", dup_count)

# 3.3 Mixed types detection (object columns usually)
mixed = []
for c in df_raw.columns:
    # Check within non-null values to avoid NaN type noise
    types = df_raw[c].dropna().map(type).nunique()
    if types > 1:
        mixed.append(c)
print("Columns with mixed Python types:", mixed if mixed else "None")

# 3.4 Candidate 'irrelevant/leakage' columns (heuristics)
leakage_keywords = ["profit", "revenue", "margin", "net_income", "ebit", "earnings", "company"]
leakage_cols = [c for c in df_raw.columns if any(k in str(c).lower() for k in leakage_keywords)]
print("⚠️ Potential leakage/irrelevant columns (review):", leakage_cols if leakage_cols else "None")


## Outliers: Why you saw “Company Profit”

Your earlier code plotted *every numeric column*, so if a column like `Company_Profit` is numeric it gets included automatically.

For **customer purchase prediction**, company-level profit often behaves like:
- **not customer behavior**, and/or
- **data leakage** (information you wouldn't know at the time of purchase)

So below we:
1) Identify **customer-relevant numeric columns** (exclude obvious company/profit/revenue),
2) Plot outliers only for those.


In [ ]:
# 3.5 Outlier check (only customer-relevant numeric columns)
num_cols = df_raw.select_dtypes(include=[np.number]).columns.tolist()

# Exclude likely leakage/company finance columns from outlier charts
exclude_outlier_keywords = ["profit", "revenue", "margin", "net_income", "earnings", "ebit", "company"]
num_cols_focus = [c for c in num_cols if not any(k in str(c).lower() for k in exclude_outlier_keywords)]

print("Numeric columns (all):", num_cols)
print("Numeric columns (outlier plots focus):", num_cols_focus)

# Boxplots for up to 12 focused numeric columns
for c in num_cols_focus[:12]:
    plt.figure()
    sns.boxplot(x=df_raw[c])
    plt.title(f"Outlier Check (Focused): {c}")
    plt.tight_layout()
    plt.show()


In [ ]:
# ============================================================
# 4) Preprocessing (with Decision Log)
# ============================================================
DECISIONS = []

def log(msg: str):
    DECISIONS.append(msg)
    print("•", msg)

def normalize_missing_strings(s: pd.Series) -> pd.Series:
    s = s.astype(str).str.strip()
    s = s.replace({
        "": np.nan, " ": np.nan,
        "nan": np.nan, "NaN": np.nan,
        "none": np.nan, "None": np.nan,
        "null": np.nan, "NULL": np.nan,
        "na": np.nan, "N/A": np.nan, "n/a": np.nan
    })
    return s

def try_parse_dates(df: pd.DataFrame) -> pd.DataFrame:
    for c in df.columns:
        if df[c].dtype == "object":
            name = str(c).lower()
            if any(k in name for k in ["date", "time", "dt", "timestamp"]):
                parsed = pd.to_datetime(df[c], errors="coerce", infer_datetime_format=True)
                if parsed.notna().mean() >= 0.5:
                    df[c] = parsed
                    log(f"Parsed datetime column: {c}")
    return df

def drop_constant_columns(df: pd.DataFrame) -> pd.DataFrame:
    nunique = df.nunique(dropna=False)
    constant_cols = nunique[nunique <= 1].index.tolist()
    if constant_cols:
        df = df.drop(columns=constant_cols)
        log(f"Dropped constant columns: {constant_cols}")
    return df

def drop_id_like_columns(df: pd.DataFrame, threshold_ratio: float = 0.98) -> pd.DataFrame:
    n = len(df)
    drop_cols = []
    for c in df.columns:
        if TARGET_COL is not None and c == TARGET_COL:
            continue
        unique_ratio = df[c].nunique(dropna=False) / max(n, 1)
        cname = str(c).lower()
        if unique_ratio >= threshold_ratio and any(k in cname for k in ["id", "uid", "key", "code", "number", "no"]):
            drop_cols.append(c)
    if drop_cols:
        df = df.drop(columns=drop_cols)
        log(f"Dropped ID-like columns: {drop_cols}")
    return df

def drop_leakage_columns(df: pd.DataFrame) -> pd.DataFrame:
    # Heuristic only—review the printed list if needed
    leakage_keywords = ["profit", "revenue", "margin", "net_income", "earnings", "ebit", "company"]
    candidates = [c for c in df.columns if any(k in str(c).lower() for k in leakage_keywords)]
    if candidates:
        # Drop them by default because this notebook is for customer purchase behavior.
        df = df.drop(columns=candidates)
        log(f"Dropped likely leakage/irrelevant company-finance columns: {candidates}")
    return df

def impute_missing(df: pd.DataFrame) -> pd.DataFrame:
    num_cols = df.select_dtypes(include=[np.number]).columns
    cat_cols = df.select_dtypes(include=["object"]).columns

    for c in num_cols:
        if df[c].isna().any():
            df[c] = df[c].fillna(df[c].median())
            log(f"Imputed numeric missing with median: {c}")

    for c in cat_cols:
        if df[c].isna().any():
            mode = df[c].mode(dropna=True)
            fill_val = mode.iloc[0] if len(mode) else "Unknown"
            df[c] = df[c].fillna(fill_val)
            log(f"Imputed categorical missing with mode: {c}")

    return df

def cap_outliers_iqr(df: pd.DataFrame, factor: float = 1.5) -> pd.DataFrame:
    num_cols = df.select_dtypes(include=[np.number]).columns
    for c in num_cols:
        x = df[c].dropna()
        if x.empty:
            continue
        q1, q3 = x.quantile(0.25), x.quantile(0.75)
        iqr = q3 - q1
        if iqr == 0:
            continue
        lo, hi = q1 - factor * iqr, q3 + factor * iqr
        df[c] = df[c].clip(lo, hi)
    log("Capped numeric outliers using IQR clipping (kept all rows).")
    return df

def preprocess(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # Normalize strings + missing tokens
    for c in df.select_dtypes(include=["object"]).columns:
        df[c] = normalize_missing_strings(df[c])
    log("Trimmed strings and normalized common missing tokens in object columns.")

    df = try_parse_dates(df)

    # Duplicates
    before = len(df)
    df = df.drop_duplicates()
    removed = before - len(df)
    if removed:
        log(f"Removed duplicate rows: {removed}")
    else:
        log("No duplicate rows removed.")

    df = drop_constant_columns(df)
    df = drop_id_like_columns(df)
    df = drop_leakage_columns(df)

    df = impute_missing(df)
    df = cap_outliers_iqr(df)

    return df

df_clean = preprocess(df_raw)

print("\n✅ Cleaned shape:", df_clean.shape)
df_clean.head(10)


In [ ]:
# Quick re-check after cleaning
print("Missing after cleaning:", int(df_clean.isna().sum().sum()))
df_clean.info()


In [ ]:
# ============================================================
# 5) Feature Engineering
# ============================================================
def feature_engineering(df: pd.DataFrame, target_col: str = None) -> pd.DataFrame:
    out = df.copy()

    # Datetime features
    dt_cols = out.select_dtypes(include=["datetime64[ns]"]).columns.tolist()
    for c in dt_cols:
        out[f"{c}_year"] = out[c].dt.year
        out[f"{c}_month"] = out[c].dt.month
        out[f"{c}_day"] = out[c].dt.day
        out[f"{c}_dayofweek"] = out[c].dt.dayofweek
        out[f"{c}_is_weekend"] = out[c].dt.dayofweek.isin([5, 6]).astype(int)
    if dt_cols:
        log(f"Created datetime features from: {dt_cols}")

    # Numeric log transforms (helps skew)
    num_cols = out.select_dtypes(include=[np.number]).columns.tolist()
    if target_col in num_cols:
        num_cols.remove(target_col)
    for c in num_cols:
        if out[c].min() >= 0:
            out[f"log1p_{c}"] = np.log1p(out[c])
    if num_cols:
        log("Created log1p features for non-negative numeric columns.")

    # Frequency encoding for categoricals (strong baseline feature)
    cat_cols = out.select_dtypes(include=["object"]).columns.tolist()
    if target_col in cat_cols:
        cat_cols.remove(target_col)
    for c in cat_cols:
        freq = out[c].value_counts(dropna=False) / len(out)
        out[f"{c}_freq"] = out[c].map(freq)
    if cat_cols:
        log("Created frequency-encoding features for categorical columns.")

    return out

df_feat = feature_engineering(df_clean, target_col=TARGET_COL)
print("✅ Features shape:", df_feat.shape)
df_feat.head(10)


In [ ]:
# ============================================================
# 6) ML-ready Encoding (One-Hot)
# ============================================================
def make_ml_ready(df: pd.DataFrame, target_col: str = None) -> pd.DataFrame:
    out = df.copy()
    y = None
    if target_col is not None and target_col in out.columns:
        y = out[target_col]
        out = out.drop(columns=[target_col])

    cat_cols = out.select_dtypes(include=["object"]).columns.tolist()
    out_enc = pd.get_dummies(out, columns=cat_cols, drop_first=True)

    if y is not None:
        out_enc[target_col] = y.values

    return out_enc

df_ml = make_ml_ready(df_feat, target_col=TARGET_COL)
print("✅ ML-ready shape:", df_ml.shape)
df_ml.head(10)


In [ ]:
# ============================================================
# 7) Save Outputs + Decision Log
# ============================================================
df_clean.to_csv(CLEAN_PATH, index=False)
df_ml.to_csv(FEATURES_PATH, index=False)

with open(REPORT_PATH, "w", encoding="utf-8") as f:
    f.write("\n".join(DECISIONS))

print("✅ Saved:")
print(" - Cleaned CSV   :", CLEAN_PATH)
print(" - Features CSV  :", FEATURES_PATH)
print(" - Decision Log  :", REPORT_PATH)

print("\n--- Decision Log Preview ---")
for line in DECISIONS[:20]:
    print("•", line)
